In [430]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
import np_utils
import numpy as np

## Load Data

In [431]:
df = pd.read_csv("malgenome")

<ipython-input-431-6867931c6ca1>:1: DtypeWarning: Columns (117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("malgenome")


## Clean Data

In [432]:
# Replace NaN Values
df = df.fillna(0)

# Convert float columns to int
int_columns = df.select_dtypes(include=['int']).columns
df[int_columns] = df[int_columns].astype(float)

In [433]:
X = df.drop(['class'], axis=1)
search_string = 'B'
result = X[X.eq(search_string).any(axis=1)]
result

,transact,bindService,onServiceConnected,ServiceConnection,android.os.Binder,READ_SMS,attachInterface,WRITE_SMS,TelephonyManager.getSubscriberId,Ljava.lang.Class.getCanonicalName,...,READ_EXTERNAL_STORAGE,Ljava.lang.Object.getClass,SET_ORIENTATION,DEVICE_POWER,EXPAND_STATUS_BAR,GET_TASKS,GLOBAL_SEARCH,GET_PACKAGE_SIZE,SET_PREFERRED_APPLICATIONS,android.intent.action.PACKAGE_CHANGED
3799,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [434]:
search_string = 'B'

# Check if the search string exists in each cell
result = X.applymap(lambda x: search_string in str(x))

# Identify the columns where the search string exists
columns_with_string = result.any()

# Get the names of the columns
columns_containing_string = columns_with_string[columns_with_string].index.tolist()

print("Columns containing the string '{}': {}".format(search_string, columns_containing_string))

Columns containing the string 'B': ['android.intent.action.NEW_OUTGOING_CALL']


In [435]:
X['android.intent.action.NEW_OUTGOING_CALL'][3799] = 0.0

<ipython-input-435-b53691ab237e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['android.intent.action.NEW_OUTGOING_CALL'][3799] = 0.0


In [436]:
X['android.intent.action.NEW_OUTGOING_CALL'][3799]

0.0

In [437]:
y = df['class']
y = y.replace({'S': 1, 'B': 0})

## Split Data into Train and Test

In [438]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [439]:
y_train = keras.utils.to_categorical(y_train,num_classes=2)
y_test = keras.utils.to_categorical(y_test,num_classes=2)

## Build the Neural Network

In [440]:
# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(215,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
])

model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_51 (Dense)            (None, 64)                13824     
                                                                 
 dense_52 (Dense)            (None, 32)                2080      
                                                                 
 dense_53 (Dense)            (None, 2)                 66        
                                                                 
Total params: 15970 (62.38 KB)
Trainable params: 15970 (62.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [441]:
model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

## Train the Neural Network

In [442]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
# Train the model
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
130/130 [==============================] - 2s 5ms/step - loss: 0.1947 - accuracy: 0.9344
Epoch 2/10
130/130 [==============================] - 0s 4ms/step - loss: 0.0447 - accuracy: 0.9879
Epoch 3/10
130/130 [==============================] - 0s 3ms/step - loss: 0.0245 - accuracy: 0.9920
Epoch 4/10
130/130 [==============================] - 0s 3ms/step - loss: 0.0137 - accuracy: 0.9969
Epoch 5/10
130/130 [==============================] - 0s 3ms/step - loss: 0.0086 - accuracy: 0.9981
Epoch 6/10
130/130 [==============================] - 0s 2ms/step - loss: 0.0061 - accuracy: 0.9990
Epoch 7/10
130/130 [==============================] - 0s 2ms/step - loss: 0.0048 - accuracy: 0.9988
Epoch 8/10
130/130 [==============================] - 0s 3ms/step - loss: 0.0034 - accuracy: 0.9993
Epoch 9/10
130/130 [==============================] - 0s 3ms/step - loss: 0.0025 - accuracy: 0.9995
Epoch 10/10
130/130 [==============================] - 0s 3ms/step - loss: 0.0018 - accuracy: 0.9995

## Evaluate the Neural Network

In [443]:
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

33/33 [==============================] - 0s 2ms/step - loss: 0.0269 - accuracy: 0.9923
Test Loss: 0.02687019295990467
Test Accuracy: 0.9922928810119629


## Test the Neural Network

In [444]:
predictions = model.predict(X_test[0:5])
predictions

1/1 [==============================] - 0s 70ms/step


array([[1.0000000e+00, 1.5611582e-13],
       [9.9999857e-01, 1.8765506e-06],
       [1.0000000e+00, 4.0047745e-19],
       [3.7234312e-07, 9.9999946e-01],
       [1.0000000e+00, 2.2850895e-10]], dtype=float32)

In [445]:
true_labels = y_test[0:5]
true_labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [446]:
arr = np.array(["B", "S"])

In [447]:
arr[0]

'B'

In [448]:
for i in range(len(predictions)):
  print("Prediction:", arr[predictions[i].argmax()], ",",
        "True Label:", arr[true_labels[i].argmax()])

Prediction: B , True Label: B
Prediction: B , True Label: B
Prediction: B , True Label: B
Prediction: S , True Label: S
Prediction: B , True Label: B


## Save  and Convert the Neural Network to a tflite model

In [449]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [450]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)